In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR") #supress warnings

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/27 15:57:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
24/11/27 15:57:04 WARN Utils: Service 'SparkUI' could not bind on

In [3]:
# Read in Data
df = spark.read.parquet("updated_dataset.parquet")

In [4]:
# Use StringIndexer for encoding the 'Severity' column
indexer = StringIndexer(inputCol="Severity_Binary", outputCol="SeverityIndex")
df = indexer.fit(df).transform(df)

In [5]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'SeverityIndex':
        continue
    elif col == "Severity_Binary":
        continue
    elif col == 'Percent_Age_65_over':
        continue
    else:
        feature_list.append(col)

In [6]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [7]:
# Undersampling
from pyspark.sql import functions as F

# Step 1: Group by 'Severity' and count occurrences
class_counts = train.groupBy("Severity_Binary").count()

# Step 2: Use PySpark's min() function to find the minimum count
min_class_size = class_counts.agg(F.min('count')).collect()[0][0]

undersampled_train_list = []

for row in class_counts.collect():
    class_label = row['Severity_Binary']
    class_size = row['count']

    if class_size > min_class_size:
        # Sample the data for this class to the size of the minimum class
        class_data = train.filter(F.col("Severity_Binary") == class_label)
        class_data_undersampled = class_data.sample(withReplacement=False, fraction=min_class_size / class_size)
    else:
        # For classes that are already at the minimum size, keep all samples
        class_data_undersampled = train.filter(F.col("Severity_Binary") == class_label)

    undersampled_train_list.append(class_data_undersampled)

# Combine all the undersampled DataFrames
undersampled_train = undersampled_train_list[0]  # start with the first one
for df in undersampled_train_list[1:]:
    undersampled_train = undersampled_train.union(df)

# Show the result
undersampled_train.show()

# Step 4: Group by 'Severity' and count the occurrences in the undersampled DataFrame
undersampled_class_counts = undersampled_train.groupBy("Severity_Binary").count()

# Show the result
undersampled_class_counts.show()

+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|Severity|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|SeverityIndex|
+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|       3|        -17|      

+---------------+-------+
|Severity_Binary|  count|
+---------------+-------+
|              1|1041410|
|              0|1040773|
+---------------+-------+



In [8]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

undersampled_train = assembler.transform(undersampled_train)
test = assembler.transform(test)

In [9]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(undersampled_train)
scaledTrainData = scalerModel.transform(undersampled_train)
scaledTestData = scalerModel.transform(test)

In [10]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='SeverityIndex',
                        featuresCol='scaledFeatures')

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

Detected number of classes: 2
Coefficients: DenseMatrix([[-0.16202166,  0.00510355,  0.15364278,  0.04115034,  0.14237387,
               0.00570116, -0.05334718, -0.00153257,  0.05563753, -0.0174447 ,
              -0.33144272, -0.18375356, -0.10916603, -0.04956626,  0.55806679,
               0.05270141, -0.01506172,  0.07460134,  0.01763486, -0.19317496,
              -0.1325262 ,  0.00311864]])
Intercept: [-4.40075950251451]


In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Compute predictions. This will append the column "prediction" to the DataFrame.
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

# Use BinaryClassificationEvaluator for binary classification
evaluator = BinaryClassificationEvaluator(labelCol='SeverityIndex', rawPredictionCol='prediction', metricName='areaUnderROC')

# Evaluate accuracy using area under ROC (or you can use 'areaUnderPR' for Precision-Recall)
roc_auc = evaluator.evaluate(lrPred, {evaluator.metricName: "areaUnderROC"})
print(f'Area Under ROC: {roc_auc}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
+----------+



Area Under ROC: 0.6411213743182302


In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

# Assuming `lrPred` is your predictions DataFrame
evaluator = MulticlassClassificationEvaluator(
    labelCol="Severity_Binary",
    predictionCol="prediction",
    metricName='truePositiveRateByLabel', 
    metricLabel=1.0
)

recall = evaluator.evaluate(lrPred)
print(f"Recall for label 1.0: {recall}")

accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
+----------+



Recall for label 1.0: 0.5680065847402473


Accuracy: 0.6870651904839137


In [13]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
#recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'truePositiveRateByLabel'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
#print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Precision: 0.7734207678803086
F1 Score: 0.716414239532677


In [14]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN Severity_Binary = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('Severity_Binary').agg(
    (F.sum('is_correct') / F.count('Severity_Binary')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()

+---------------+------------------+
|Severity_Binary|          accuracy|
+---------------+------------------+
|              1|0.5680065847402473|
|              0|0.7142361638962132|
+---------------+------------------+



In [15]:
# Calculate metrics by class
labels = [row['Severity_Binary'] for row in lrPred.select('Severity_Binary').distinct().orderBy('Severity_Binary').collect()]

metrics = {}
for label in labels:
    # Filter predictions for the current label
    true_positive = lrPred.filter((F.col('Severity_Binary') == label) & (F.col('prediction') == label)).count()
    false_positive = lrPred.filter((F.col('Severity_Binary') != label) & (F.col('prediction') == label)).count()
    false_negative = lrPred.filter((F.col('Severity_Binary') == label) & (F.col('prediction') != label)).count()
    true_negative = lrPred.filter((F.col('Severity_Binary') != label) & (F.col('prediction') != label)).count()

    # Precision, Recall, and F1 Score
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # Store metrics
    metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

# Print metrics for each class
for label, metric in metrics.items():
    print(f"Class {label} - Precision: {metric['precision']:.4f}, Recall: {metric['recall']:.4f}, F1 Score: {metric['f1_score']:.4f}")

Class 0 - Precision: 0.8787, Recall: 0.7142, F1 Score: 0.7880
Class 1 - Precision: 0.3121, Recall: 0.5680, F1 Score: 0.4028
